In [2]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
if __name__ == "__main__":
    sys.path.append("../../../")
from notebooks_src.postprocessing.utils import get_int_tensor_shape
import tensorflow as tf
slim=tf.contrib.slim
from notebooks_src.losses.utils import abs_smooth as smooth_L1

importing Jupyter notebook from ../../../notebooks_src/postprocessing/utils.ipynb
importing Jupyter notebook from ../../../notebooks_src/losses/utils.ipynb
box_encode_decode_configs
box_encode_decode_configs
tensorboard_configs
fit_configs
labels_configs
load_data_configs
losses_configs
metrics_configs
models_configs
optimizers_configs
tensorboard_configs
fit_configs
labels_configs
load_data_configs
losses_configs
metrics_configs
models_configs
optimizers_configs
